In [2]:
from marshmallow import Schema, fields, post_load, INCLUDE, EXCLUDE
import json

### Dealing with kebabcase -> snakecase

In [3]:
def kebabcase(s):
    return s.replace('_', '-')

class KebabCaseSchema(Schema):
    """
    Schema that uses kebab-case for its external representation
    and snake-case for its internal representation.
    """
    class Meta:
        unknown = EXCLUDE
        
    def on_bind_field(self, field_name, field_obj):
        field_obj.data_key = kebabcase(field_obj.data_key or field_name)

### Standard marshmallow

In [4]:
class Item:
    def __init__(self, product_type, options, artist_markup, quantity, **kwargs):
        self.product_type = product_type
        self.options = options
        self.artist_markup = artist_markup
        self.quantity = quantity
        
    def __repr__(self):
        return f"<Item: {self.product_type}>"

class ItemSchema(KebabCaseSchema):
    """
    A schema for items as defined within cart.schema.json
    Converts kebab case to snake case
    """
    class Meta:
        unknown = EXCLUDE
        
    product_type = fields.Str(required=True)
    options = fields.Dict(required=True)
    artist_markup = fields.Int(reuqired=True)
    quantity = fields.Int(required=True)

    @post_load
    def load_item(self, data, **kwargs):
        return Item(**data)


In [5]:
item = {
    "product-type": "shirt", 
    "options": {"key": "value"}, 
    "artist-markup": "5",
    "quantity": 2
}
item_json = json.dumps(item)

In [6]:
item = ItemSchema().loads(item_json, unknown=INCLUDE)

### Marshmallow dataclass

Allows for most DRY code via not requiring post_load step

In [7]:
import marshmallow_dataclass
from typing import Dict, Any

In [8]:
@marshmallow_dataclass.dataclass(base_schema=KebabCaseSchema)
class Item:     
    product_type: str
    options: Dict[Any, Any]
    artist_markup: int
    quantity: int

In [9]:
item = {
    "product-type": "shirt", 
    "options": {"key": "value"}, 
    "artist-markup": "5",
    "quantity": 2,
    "other": 3
}
item_json = json.dumps(item)

In [10]:
item = Item.Schema().loads(item_json)

### Loading demo data carts

In [11]:
demo_files = ["cart-4560.json", "cart-9363.json", "cart-9500.json", "cart-11356.json"]

In [12]:
for file in demo_files:
    with open(f"../examples/{file}", "r") as f:
        data = json.load(f)
        
    items = Item.Schema(many=True).load(data)
    print(f'File: {file} \nItems: {items}\n')

File: cart-4560.json 
Items: [Item(product_type='hoodie', options={'size': 'small', 'colour': 'white', 'print-location': 'front'}, artist_markup=20, quantity=1)]

File: cart-9363.json 
Items: [Item(product_type='hoodie', options={'size': 'small', 'colour': 'dark', 'print-location': 'front'}, artist_markup=20, quantity=2), Item(product_type='sticker', options={'size': 'small'}, artist_markup=10, quantity=1)]

File: cart-9500.json 
Items: [Item(product_type='hoodie', options={'size': 'small', 'colour': 'white', 'print-location': 'front'}, artist_markup=20, quantity=1), Item(product_type='hoodie', options={'size': 'small', 'colour': 'dark', 'print-location': 'front'}, artist_markup=30, quantity=1)]

File: cart-11356.json 
Items: [Item(product_type='hoodie', options={'size': 'xl', 'colour': 'dark', 'print-location': 'back'}, artist_markup=30, quantity=2)]



In [25]:
import itertools

In [51]:
lists = [['hello'], [3,'5'], ['j', 'k']]
tuples = list(itertools.product(*lists))
'_'.join(str(key) for key in tuples[0])

'hello_3_j'

In [49]:
tuples

[('hello', 'a', 'j'),
 ('hello', 'a', 'k'),
 ('hello', 'b', 'j'),
 ('hello', 'b', 'k')]

In [35]:
a = {'yyy': 1, 'hi': [1,2,3], 'bye': [4,5,6], 'zzz' :5}

In [36]:
sorted(a.keys())

['bye', 'hi', 'yyy', 'zzz']

In [24]:
for key, value in a.items():
    print(key, value)

hi [1, 2, 3]
bye [4, 5, 6]
